In [ ]:
import numpy as np
import pandas as pd
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import requests
from xml.etree import ElementTree
import numpy as np
import folium
import folium.plugins
# Used Google OR-tools library for VRP algorithm solving
# Used folium for visualization

In [ ]:
# distance matrix 생성
# 참고한 코드: https://stackoverflow.com/questions/19413259/efficient-way-to-calculate-distance-matrix-given-latitude-and-longitude-data-in

def distance_on_sphere_numpy(coordinate_array):
    """
    Compute a distance matrix of the coordinates using a spherical metric.
    :param coordinate_array: numpy.ndarray with shape (n,2); latitude is in 1st col, longitude in 2nd.
    :returns distance_mat: numpy.ndarray with shape (n, n) containing distance in km between coords.
    """
    # Radius of the earth in km (GRS 80-Ellipsoid)
    EARTH_RADIUS = 6371.007176 

    # Unpacking coordinates
    latitudes = coordinate_array[:, 0]
    longitudes = coordinate_array[:, 1]

    # Convert latitude and longitude to spherical coordinates in radians.
    degrees_to_radians = np.pi/180.0
    phi_values = (90.0 - latitudes)*degrees_to_radians
    theta_values = longitudes*degrees_to_radians

    # Expand phi_values and theta_values into grids
    theta_1, theta_2 = np.meshgrid(theta_values, theta_values)
    theta_diff_mat = theta_1 - theta_2

    phi_1, phi_2 = np.meshgrid(phi_values, phi_values)

    # Compute spherical distance from spherical coordinates
    angle = (np.sin(phi_1) * np.sin(phi_2) * np.cos(theta_diff_mat) + 
           np.cos(phi_1) * np.cos(phi_2))
    arc = np.arccos(angle)

    # Multiply by earth's radius to obtain distance in km
    return arc * EARTH_RADIUS

In [ ]:
on_79 = pd.read_csv("point_79_on.csv")
on_79.head()

,Unnamed: 0,이비카드정류장ID,정류장이름,위도,경도,횟수
0,316,4197607.0,서동탄역신일해피트리(중),37.199500,127.052483,317.0
1,808,4197590.0,솔빛마을.서해.쌍용아파트,37.195817,127.069633,304.0
2,1143,4116691.0,안녕농협삼거리,37.205217,127.014700,335.0
3,1649,4117227.0,수원대학교,37.213750,126.980150,506.0
4,2030,4197467.0,한빛마을(중),37.207567,127.069200,381.0


In [ ]:
latlong = on_79[['위도', '경도']]
latlong = latlong.to_numpy()

In [ ]:
dis_matrix = distance_on_sphere_numpy(latlong)
# discover the latitude and longitued through latlong
# made the distance matrix using latlong

/home/yeeun.song.conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in arccos


In [ ]:
dis_matrix

array([[0.00000000e+00, 1.57327132e+00, 3.40620938e+00, ...,
        2.54804636e+00, 5.39239804e+00, 2.63521791e+00],
       [1.57327132e+00, 0.00000000e+00, 4.97642612e+00, ...,
        1.79850429e+00, 3.88494576e+00, 1.24418755e+00],
       [3.40620938e+00, 4.97642612e+00,            nan, ...,
        5.60227580e+00, 8.69169277e+00, 5.92585791e+00],
       ...,
       [2.54804636e+00, 1.79850429e+00, 5.60227580e+00, ...,
        0.00000000e+00, 4.60095056e+00, 2.69658340e+00],
       [5.39239804e+00, 3.88494576e+00, 8.69169277e+00, ...,
        4.60095056e+00, 0.00000000e+00, 2.77149823e+00],
       [2.63521791e+00, 1.24418755e+00, 5.92585791e+00, ...,
        2.69658340e+00, 2.77149823e+00, 9.49354049e-05]])

In [ ]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dis_matrix
    data['num_vehicles'] = 4
    data['depot'] = 15
    return data
# 참고한 코드: https://developers.google.com/optimization/routing/vrp#distance_matrix_api
# Google OR-Tools
# num_vehicles: 몇대의 운송수단으로 할건지
# depot: 시작하고 끝나는 노드, 중간쯔음의 노드로 결정

In [ ]:
# for printing the vrp route order
# not for visualiztion, for listing the order of VRP routes

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        # print(vehicle_id)
        while not routing.IsEnd(index):
            
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            
            index = solution.Value(routing.NextVar(index))
            # print(index)
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))


In [ ]:
# function for getting list of VRP results, for visualization uses

def get_lists(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    data_numpy = []
    # 2 dimensional list
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        
        
        
        data_list = []
        # 1 dimensional list, list for just one vehicle
            
        route_distance = 0
        while not routing.IsEnd(index):
            
            
            previous_index = index
            
            
            data_list.append(index)
            index = solution.Value(routing.NextVar(index))
            # print(index)
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        #plan_output += '{}\n'.format(manager.IndexToNode(index))
        #plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        #print(plan_output)
        #max_route_distance = max(route_distance, max_route_distance)
        
        data_numpy.append(data_list)
    return data_numpy

    #print('Maximum of the route distances: {}m'.format(max_route_distance))
    # print(data_numpy)
    

In [ ]:

"""Solve the CVRP problem."""
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    3000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    list_route = get_lists(data, manager, routing, solution)
    print_solution(data, manager, routing, solution)



Route for vehicle 0:
 15 ->  33 ->  43 ->  37 ->  36 ->  10 ->  24 ->  31 ->  28 ->  3 ->  22 ->  17 ->  35 ->  5 ->  16 ->  38 ->  23 ->  2 ->  29 -> 15
Distance of the route: 11m

Route for vehicle 1:
 15 ->  14 ->  26 ->  13 ->  25 ->  30 ->  21 ->  8 ->  20 ->  32 ->  19 ->  47 ->  12 ->  42 ->  44 ->  40 ->  39 ->  41 ->  7 ->  46 ->  27 ->  4 ->  0 -> 15
Distance of the route: 11m

Route for vehicle 2:
 15 ->  11 -> 15
Distance of the route: 0m

Route for vehicle 3:
 15 ->  18 ->  9 ->  49 ->  48 ->  45 ->  6 ->  1 ->  34 -> 15
Distance of the route: 9m

Maximum of the route distances: 11m


In [ ]:
list_route
data['depot']
# 처음에 시작하는 정류소 index

15

In [ ]:
for i in range(0,4):
    list_route[i][0] = data['depot']
    list_route[i].append(data['depot'])
# list value 정리

In [ ]:
list_route
# list_route 결과가 고르지 않다는 것을 확인
# capacity optimization을 두는 CVRP를 다음에 시도해야겠다.
# 참고: https://developers.google.com/optimization/routing/cvrp

[[15, 33, 43, 37, 36, 10, 24, 31, 28, 3, 22, 17, 35, 5, 16, 38, 23, 2, 29, 15],
 [15,
  14,
  26,
  13,
  25,
  30,
  21,
  8,
  20,
  32,
  19,
  47,
  12,
  42,
  44,
  40,
  39,
  41,
  7,
  46,
  27,
  4,
  0,
  15],
 [15, 11, 15],
 [15, 18, 9, 49, 48, 45, 6, 1, 34, 15]]

In [ ]:
df1 = on_79.reindex(list_route[0])
df2 = on_79.reindex(list_route[1])
df3 = on_79.reindex(list_route[2])
df4 = on_79.reindex(list_route[3])

# 각각의 노선 정류소 dataframe

In [ ]:
latitude = 37.136881
longitude = 127.036171
m = folium.Map(location=[latitude, longitude], zoom_start=12)

#folium for visualization

In [ ]:
locations = df1[['위도', '경도']]
locationlist1 = locations.values.tolist()
len(locationlist1)

i=1

for point in range(0, len(locationlist1)):
    # folium.Marker(locationlist[point], replaced['정류장이름'][point]).add_to(m)
    folium.Marker(locationlist1[point], i).add_to(m)
    i = i+1
m

# draw lines
#add lines
folium.PolyLine(locationlist1, color="red", weight=2.5, opacity=1).add_to(m)

In [ ]:
m

In [ ]:
locations = df2[['위도', '경도']]
locationlist2 = locations.values.tolist()
len(locationlist2)

i=1

for point in range(0, len(locationlist2)):
    # folium.Marker(locationlist[point], replaced['정류장이름'][point]).add_to(m)
    folium.Marker(locationlist2[point], i).add_to(m)
    i = i+1
m

# draw lines
#add lines
folium.PolyLine(locationlist2, color="purple", weight=2.5, opacity=1).add_to(m)

In [ ]:
m

In [ ]:
locations = df3[['위도', '경도']]
locationlist3 = locations.values.tolist()
len(locationlist3)

i=1

for point in range(0, len(locationlist3)):
    # folium.Marker(locationlist[point], replaced['정류장이름'][point]).add_to(m)
    folium.Marker(locationlist3[point], i).add_to(m)
    i = i+1
m

# draw lines
#add lines
folium.PolyLine(locationlist3, color="yellow", weight=2.5, opacity=1).add_to(m)

In [ ]:
m

In [ ]:
locations = df4[['위도', '경도']]
locationlist4 = locations.values.tolist()
len(locationlist4)

i=1

for point in range(0, len(locationlist4)):
    folium.Marker(locationlist4[point], i).add_to(m)
    i = i+1
m

# draw lines
# add lines
folium.PolyLine(locationlist4, color="green", weight=2.5, opacity=1).add_to(m)

In [ ]:
m